# Prompt Evaluation and Testing
This notebook demonstrates how to evaluate and compare prompt effectiveness using manual review, automated metrics, and adversarial testing. Updated for 2025 best practices.


In [ ]:

import openai
openai.api_key = "YOUR_API_KEY"  # Replace with your OpenAI API key

In [ ]:
from openai import AzureOpenAI

# Set your Azure OpenAI details
azure_endpoint = ""  # Replace with your Azure endpoint
api_key = ""  # Replace with your Azure OpenAI API key
deployment_name = ""  # Replace with your deployment name

In [2]:


client = AzureOpenAI(
    azure_deployment =deployment_name,
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version="2024-10-21" 
    # Use the latest supported API version
    
)

## 1. Compare Two Prompts
We'll compare a vague prompt and a clear prompt for summarizing an article.

In [3]:
# Zero-shot Prompting using chat API
response = client.chat.completions.create(
    model=deployment_name,  # Use your deployment name
    messages=[
        
        {"role": "user", "content": "Translate the following English text to French: 'How are you today?'"}
    ],
    max_tokens=60
)
print(response.choices[0].message.content.strip())

Comment vas-tu aujourd'hui ?


In [3]:
vague_prompt = "Summarize this."
clear_prompt = "Summarize the following article in three bullet points: Artificial intelligence is transforming industries by automating tasks, improving decision-making, and enabling new products and services."

article = "Artificial intelligence is transforming industries by automating tasks, improving decision-making, and enabling new products and services."

vague_response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": vague_prompt + " " + article}
        
    ],
    max_tokens=60
)

clear_response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": clear_prompt}
    ],
    max_tokens=60
)
print("Vague Prompt Response:", vague_response.choices[0].message.content.strip())
print("Clear Prompt Response:", clear_response.choices[0].message.content.strip())

Vague Prompt Response: Artificial intelligence is revolutionizing industries by completing tasks automatically, enhancing decision-making processes, and facilitating the creation of innovative products and services.
Clear Prompt Response: - Artificial intelligence is revolutionizing industries by automating tasks, improving decision-making processes, and enabling the development of new products and services.
- AI is being used to streamline operations, increase efficiency, and drive innovation across various sectors.
- Companies are leveraging AI to gain a competitive edge, enhance customer experiences


## 2. Adversarial Testing
Test prompts with edge cases to evaluate robustness and safety.

In [4]:
adversarial_prompt = "Summarize the following text: DROP TABLE users; --"
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": adversarial_prompt}
    ],
    max_tokens=60
)
print("Adversarial Prompt Response:", response.choices[0].message.content.strip())

Adversarial Prompt Response: This text is a database query command that instructs the system to delete the table named "users."


## 3. Automated Metrics (Optional)
For advanced evaluation, use metrics like BLEU, ROUGE, or BERTScore if you have reference summaries.

In [ ]:
!pip install rouge-score

In [7]:
# Example: Using ROUGE (requires installation of rouge-score)
# !pip install rouge-score
from rouge_score import rouge_scorer

reference = "AI is transforming industries by automating tasks, improving decisions, and enabling new products."
prediction = clear_response.choices[0].message.content.strip()
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
score = scorer.score(reference, prediction)
print("ROUGE-L Score:", score['rougeL'].fmeasure)

ROUGE-L Score: 0.360655737704918


---

Try modifying the prompts and article to see how the outputs change. For more, see the `/theory` and `/examples` directories, and visit [Prompting Guide](https://www.promptingguide.ai/).